In [1]:
print('hello world')

hello world


In [5]:
import numpy as np
import pandas as pd
from selenium import webdriver

In [7]:
driver = webdriver.Chrome(r"/Users/jarkrunglerdkriangkrai/Github/setscraping/chromedriver")
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="47446e4ff9bfe360b4a2be04223e8e80")>